In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
import string
import plotly
import plotly.graph_objects as go
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
#Reading the dataset. 
df = pd.read_csv('complaints.csv')

In [ ]:
#Data frame before performing data cleanup.
df

##### Since we are only focused on the products and complaint narratives, we will re-create the data frame using only the 'Product' and 'Consumer complaint narrative' attributes.

In [ ]:
df = df[['Product' , 'Consumer complaint narrative']]

In [ ]:
#Rename attributes of dataframe for simplicity.
df = df.rename(columns={'Product' : "Product",
                       'Consumer complaint narrative' : "Complaint"})

In [ ]:
#Data frame after renaming attributes.
df

In [ ]:
df.groupby('Product').count()

##### Many of the products of the same category can be merged together into a single product category.

In [ ]:
#Clean up data by renaming similar products into one category of products.
df['Product'].replace({'Bank account or service': 'Banking Services',
                       'Checking or savings account' : 'Banking Services',
                       'Consumer Loan' : 'Loans',
                       'Credit card' : 'Credit/Prepaid Cards',
                       'Credit card or prepaid card' : 'Credit/Prepaid Cards',
                       'Credit reporting' : 'Credit Reporting and Services',
                       'Credit reporting, credit repair services, or other personal consumer reports' : 'Credit Reporting and Services',
                       'Debt collection' : 'Debt Collection',
                       'Money transfer, virtual currency, or money service' : 'Banking Services',
                       'Money transfers' : 'Banking Services',
                       'Mortgage' : 'Mortgages',
                       'Other financial service' : 'Banking Services',
                       'Payday loan' : 'Loans',
                       'Payday loan, title loan, or personal loan' : 'Loans',
                       'Prepaid card' : 'Credit/Prepaid Cards',
                       'Student loan' : 'Loans',
                       'Vehicle loan or lease' : 'Loans',
                       'Virtual currency' : 'Crypto Currency'}, inplace=True)

#Refined products and their counts.
df.groupby('Product').count()

In [ ]:
#Data frame with refined product categories.
df

### Cleanup data frame of null values.

In [ ]:
#Create new df to hold only non-null values of Consumer complaint narratives.
df = df[pd.notnull(df['Complaint'])]

In [ ]:
stop_words = stopwords.words('english') + list(string.punctuation)
stop_words += ["''", '""', '...', '``', '--', 'XXXX']

In [ ]:
#Tokenize complaint data and remove stop words from complaint narrative.
def processComplaint(comp):
    tokens = nltk.word_tokenize(comp)
    removed_stop_words = [token.lower() for token in tokens if token.lower() not in stop_words]
    new_removed_stop_words = [word for word in removed_stop_words if word.isalpha()]
    
    return new_removed_stop_words

In [ ]:
#Link words together.
def linkWords(words):
    linked_words = ''
    
    for w in words:
        linked_words += w + ' '
    
    return linked_words.strip()

In [ ]:
lm = WordNetLemmatizer()

In [ ]:
#Group variants of the same word and merge complaints.
def lemLink(words):
    words = [word for word in words if word is not np.nan]
    
    lem_list = []
    
    for idx, word in enumerate(words):
        lem_list.append(lm.lemmatize(word))
    
    linked_str = linkWords(lem_list)
    
    return linked_str

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
for i in range(len(df)):
    processed_complaints = processComplaint(df['Complaint'].iloc[i])
    complaint = lemLink(processed_complaints)
    df['Complaint'].iloc[i] = complaint
    
    if i % 5000 == 0:
        print(f'Processed row #: {i}')